<a href="https://colab.research.google.com/github/JhaAyushCanCode/Quantum-Classical_Hybrid_ML/blob/main/Failed_iterations/QML_QML_Iteration_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Quantum-Classical Hybrid Neural Network (Pure PennyLane, GoEmotions Dataset, Conflict-Free)

# Step 1: Environment Setup (Google Colab)
# Uninstall incompatible JAX versions and reinstall a compatible one
!pip uninstall -y jax jaxlib
!pip install jax==0.4.28 jaxlib==0.4.28 --quiet
!pip install pennylane seaborn tensorflow-datasets scikit-learn==1.6.1 --upgrade --quiet

In [2]:
# Step 2: Import Required Libraries
import pennylane as qml
from pennylane import numpy as np
import numpy as onp
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Step 3: Load GoEmotions Dataset
print("Loading GoEmotions dataset...")
dataset, info = tfds.load('goemotions', with_info=True)
train_dataset = dataset['train']

# Define emotion labels in the correct order
emotion_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
    'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment',
    'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral',
    'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise'
]

texts, labels = [], []
for example in tfds.as_numpy(train_dataset):
    texts.append(example['comment_text'].decode('utf-8'))

    for idx, label in enumerate(emotion_labels):
        if example[label]:
            labels.append(idx)
            break
    else:
        labels.append(20)

n_classes = 28

Loading GoEmotions dataset...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/goemotions/incomplete.XZ7GT4_0.1.0/goemotions-validation.tfrecord*...:   0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/goemotions/incomplete.XZ7GT4_0.1.0/goemotions-test.tfrecord*...:   0%|    …

Dataset goemotions downloaded and prepared to /root/tensorflow_datasets/goemotions/0.1.0. Subsequent calls will reuse this data.


In [4]:
# Step 4: Text Vectorization (TF-IDF)
print("Vectorizing text using TF-IDF...")
vectorizer = TfidfVectorizer(max_features=16)
X = vectorizer.fit_transform(texts).toarray()
y = onp.array(labels)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_train_onehot = onp.eye(n_classes)[y_train]
y_test_onehot = onp.eye(n_classes)[y_test]

Vectorizing text using TF-IDF...


In [5]:
# Step 5: Quantum Device Setup
n_qubits = 16
dev = qml.device("default.qubit", wires=n_qubits)

In [6]:
# Step 6: Build Deeper Quantum Circuit
@qml.qnode(dev, interface='autograd')
def quantum_circuit(inputs, weights, n_layers):
    for i in range(n_qubits):
        qml.RY(inputs[i], wires=i)

    for layer in range(n_layers):
        for i in range(n_qubits):
            qml.RY(weights[layer, i, 0], wires=i)
            qml.RZ(weights[layer, i, 1], wires=i)
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])
        qml.CNOT(wires=[n_qubits - 1, 0])

    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [7]:
# Step 7: Prediction Logic
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / np.sum(e_x, axis=1, keepdims=True)

def predict(X, weights, W_output, b_output, n_layers):
    preds = [quantum_circuit(x, weights, n_layers) for x in X]
    logits = np.array(preds)
    probs = softmax(logits @ W_output + b_output)
    return probs

In [8]:
# Step 8: Parameter Initialization
np.random.seed(42)
n_layers = 5
weights = 0.01 * np.random.randn(n_layers, n_qubits, 2, requires_grad=True)
W_output = 0.01 * np.random.randn(n_qubits, n_classes)
b_output = 0.01 * np.random.randn(n_classes)

In [ ]:
# Step 9: Training Loop (Pure Autograd, Fixed Layers)
opt = qml.AdamOptimizer(stepsize=0.01)
epochs = 100
batch_size = 32
accuracy_history = []

for epoch in range(epochs):
    batch_index = onp.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[batch_index]
    y_batch = y_train_onehot[batch_index]

    def cost(weights):
        preds = predict(X_batch, weights, W_output, b_output, n_layers)
        return -np.mean(np.sum(y_batch * np.log(preds + 1e-10), axis=1))

    weights = opt.step(cost, weights)

    if epoch % 5 == 0:
        y_pred = predict(X_test, weights, W_output, b_output, n_layers)
        acc = accuracy_score(onp.array(y_test), onp.argmax(y_pred, axis=1))
        accuracy_history.append(acc)
        print(f"Epoch {epoch}: Test Accuracy = {acc:.2f}")

Epoch 0: Test Accuracy = 0.05
Epoch 5: Test Accuracy = 0.05
Epoch 10: Test Accuracy = 0.05
Epoch 15: Test Accuracy = 0.04
